In [1]:
import pandas as pd
import sklearn
import numpy as np

In [2]:
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, v_measure_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning

from sklearn import tree, naive_bayes
from sklearn.utils._testing import ignore_warnings

In [3]:
sklearn.__version__

'0.24.2'

# Предобработка данных

## Загрузка данных

### Классификация

In [4]:
df = pd.read_csv('../data/bank_additional_preprocessed.csv', sep=';')

In [5]:
df

,age,campaign,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,job_admin.,...,poutcome_nonexistent,poutcome_success,previous_0,previous_1,previous_2,previous_3,previous_4,previous_5,previous_6,previous_7
0,56,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,57,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,37,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,40,1,999,1.1,93.994,-36.4,4.857,5191.0,0,1,...,1,0,1,0,0,0,0,0,0,0
4,56,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,999,-1.1,94.767,-50.8,1.028,4963.6,1,0,...,1,0,1,0,0,0,0,0,0,0
41184,46,1,999,-1.1,94.767,-50.8,1.028,4963.6,0,0,...,1,0,1,0,0,0,0,0,0,0
41185,56,2,999,-1.1,94.767,-50.8,1.028,4963.6,0,0,...,1,0,1,0,0,0,0,0,0,0
41186,44,1,999,-1.1,94.767,-50.8,1.028,4963.6,1,0,...,1,0,1,0,0,0,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  int64  
 1   campaign                       41188 non-null  int64  
 2   pdays                          41188 non-null  int64  
 3   emp.var.rate                   41188 non-null  float64
 4   cons.price.idx                 41188 non-null  float64
 5   cons.conf.idx                  41188 non-null  float64
 6   euribor3m                      41188 non-null  float64
 7   nr.employed                    41188 non-null  float64
 8   y                              41188 non-null  int64  
 9   job_admin.                     41188 non-null  int64  
 10  job_blue-collar                41188 non-null  int64  
 11  job_entrepreneur               41188 non-null  int64  
 12  job_housemaid                  41188 non-null 

Посмотрим на сбалансированность признаков.

In [7]:
labels, samples = np.unique(df.y, return_counts=True)
labels, samples

(array([0, 1], dtype=int64), array([36548,  4640], dtype=int64))

Как мы видим, данные несбалансированы: данных класса 0 приблизительно в 10 раз больше, чем данных класса 1.

При таком дисбалансе при downsampling потеряется большое число информации, поэтому выполним upsampling.

Для того, чтобы данные из обучающей выборки не попали в тестовую, выполним разбиение на обучение и тест

In [8]:
def upsample(df_big, df_small, verbose=False):
    if verbose:
        print('Upsample from {} to {}'.format(len(df_small), len(df_big)))
    df_small_copy = df_small.copy()
    i = len(df_small)
    while i < len(df_big):
        if verbose:
            print('GET i: {}'.format(i))
        if i + len(df_small_copy) < len(df_big):
            df_small = df_small.append(df_small_copy)
            i += len(df_small_copy)
        else:
            df_small = df_small.append(df_small_copy.loc[np.random.permutation(len(df_small_copy))].loc[i % len(df_small_copy) : (len(df_big) - 1) % len(df_small_copy)])
            i = len(df_small)
    return df_small

In [9]:
def downsample(df_big, df_small, verbose=False):
    if verbose:
        print('Downsample from {} to {}'.format(len(df_big), len(df_small)))
    return df_big.loc[np.random.permutation(len(df_big))[:len(df_small)]]

In [10]:
'''
# Upsample
df_0 = df.loc[df.y == 0].reset_index(drop=True)
df_0_train, df_0_test = train_test_split(df_0, test_size=0.2)

df_1 = df.loc[df.y == 1].reset_index(drop=True)
df_1_train, df_1_test = train_test_split(df_1, test_size=0.2)

df_0_train = df_0_train.reset_index(drop=True)
df_1_train = df_1_train.reset_index(drop=True)

df_0_test = df_0_test.reset_index(drop=True)
df_1_test = df_1_test.reset_index(drop=True)

df_1_train_up = upsample(df_0_train, df_1_train, verbose=True)
df_1_test_up = upsample(df_0_test, df_1_test, verbose=True)

df_train = pd.concat([df_0_train, df_1_train_up], ignore_index=True)
df_train = df_train.loc[np.random.permutation(len(df_train))]
df_test = pd.concat([df_0_test, df_1_test_up], ignore_index=True)
df_test = df_test.loc[np.random.permutation(len(df_test))]

df_sample = pd.concat([df_train, df_test], ignore_index=True)
print(df_train.shape, df_train[df_train.y == 0].shape, df_train[df_train.y == 1].shape)
'''
# Downsample
# num_obj = 1000
df_0 = df.loc[df.y == 0].reset_index(drop=True)
df_1 = df.loc[df.y == 1].reset_index(drop=True)
# df_1 = df_1.loc[np.random.permutation(len(df_1))[:num_obj]]
df_0 = downsample(df_0, df_1, verbose=True)

df_sample = pd.concat([df_0, df_1], ignore_index=True)

Downsample from 36548 to 4640


In [11]:
X = df_sample.drop(['y'], axis=1)
y = df_sample.y
X.shape, y.shape

((9280, 69), (9280,))

Для удобства обработки разделим признаки на категориальные и вещественные

In [12]:
numeric_cols = np.array(['age', 'campaign', 'pdays', 'emp.var.rate', 'cons.price.idx', 
           'cons.conf.idx', 'euribor3m', 'nr.employed'])
X_numeric = X[numeric_cols]

In [13]:
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))
X_categorical = X[categorical_cols]
for col in categorical_cols:
    X_categorical[col] = X_categorical[col].astype('string')

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [14]:
print(X_categorical.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9280 entries, 0 to 9279
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   contact_telephone              9280 non-null   string
 1   marital_divorced               9280 non-null   string
 2   poutcome_nonexistent           9280 non-null   string
 3   contact_cellular               9280 non-null   string
 4   month_jul                      9280 non-null   string
 5   day_of_week_fri                9280 non-null   string
 6   education_professional.course  9280 non-null   string
 7   day_of_week_mon                9280 non-null   string
 8   previous_1                     9280 non-null   string
 9   previous_3                     9280 non-null   string
 10  day_of_week_wed                9280 non-null   string
 11  previous_2                     9280 non-null   string
 12  month_aug                      9280 non-null   string
 13  def

Разделим данные на обучение и тест

In [15]:
X_train, X_test, X_train_cat, X_test_cat, X_train_num, X_test_num, y_train, y_test = train_test_split(X, X_categorical, X_numeric, y, test_size=0.2)

Нормализируем значения

In [16]:
scaler = StandardScaler()
scaler.fit(X_numeric)

X_train_num_sc = scaler.transform(X_train_num)
X_test_num_sc = scaler.transform(X_test_num)
X_num_sc = scaler.transform(X_numeric)

Соединяем воедино все значения

In [17]:
X_train_transform = np.hstack((X_train_num_sc, X_train_cat))
X_test_transform = np.hstack((X_test_num_sc, X_test_cat))
X_transform = np.hstack((X_num_sc, X_categorical))

Для некоторых алгоритмов введено условие неотрицательности, реализуем его

# Обучение модели

Напишем методы, которые будут обучать модель и предсказывать результат

Данный метод будет выводить на печать метрики в задаче [классификации](https://scikit-learn.org/0.20/modules/classes.html#classification-metrics) и [кластеризации](https://scikit-learn.org/0.20/modules/classes.html#clustering-metrics).

In [18]:
def print_metrics(X, y_true, y_pred, type_task='classification'):
    if type_task == 'classification':
        print('Confusion matrix:')
        print(confusion_matrix(y_true, y_pred))
        print('Classification report:')
        print(classification_report(y_true, y_pred))
    elif type_task == 'clustering':
        print('Silhouette score:')
        print(silhouette_score(X, y_pred))
        print('V measure score:')
        print(v_measure_score(y_true, y_pred))

Функция обучения модели и предсказывания значений

In [19]:
def fit_predict(X_train, X_test, y_train, y_test, type_task='classification', 
                model=tree.DecisionTreeClassifier, init_parameters={}, custom_predict=None,
                use_test=True, 
                verbose=False, draw=False, **parameters):
    
    if not use_test:
        X_test = X_train
        y_test = y_train
    if custom_predict:
        X_test, y_test, y_pred = custom_predict(X_train, X_test, y_train, y_test, type_task, model, init_parameters, verbose=verbose, draw=draw, **parameters)
    elif type_task == 'classification':
        print('Start fit model and predict values')
        classifier_model = model(**init_parameters)
        clf_model = GridSearchCV(classifier_model, parameters)
        clf_model.fit(X_train, y_train)
        # Get best model
        best_est = clf_model.best_estimator_
        if verbose:
            print('GridSearchCV: best estimator:\n{},\nwith best parameters:\n{}'.format(clf_model.best_estimator_, clf_model.best_params_))
        # Predict value
        y_pred = best_est.predict(X_test)
    elif type_task == 'clustering':
        print('Start fit model and predict values')
        classifier_model = model(**init_parameters)
        classifier_model.fit(X_train)
        # Обучаем модель
        y_pred = classifier_model.predict(X_test)
    
    if verbose:
        print_metrics(X_test, y_test, y_pred, type_task=type_task)
    if draw:
        if(X_test.shape[1] == 2):
            plot_2d(X_test[:, 0], X_test[:, 1], y_pred)
        elif(X_test.shape[1] == 3):
            plot_3d(X_test[:, 0], X_test[:, 1], X_test[:, 2], y_pred)
        else:
            while X_test.shape[1] < 3:
                print('Add useless zeros dimension to dataset')
                X_test = np.hstack((X_test, np.zeros((X_test.shape[0], 1))))
            pca = PCA(n_components=3)
            pca_3d = pca.fit_transform(X_test, y_test)

            x = pca_3d[:, 0]
            y = pca_3d[:, 1]
            z = pca_3d[:, 2]

            plot_3d(x, y, z, y_pred)

Данная функция печатает метки для различных значений отбора признаков

In [20]:
@ignore_warnings(category=ConvergenceWarning)
def print_with_selection_features(model=tree.DecisionTreeClassifier, init_parameters={}, custom_fit_predict=None, **parameters):
    if custom_fit_predict:
        fit_predict = custom_fit_predict
    %time fit_predict(X_train_transform, X_test_transform, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)

## Задание 1

# Формула Байеса

Формула Байеса позволяет «переставить причину и следствие»: по известному факту события вычислить вероятность того, что оно было вызвано данной причиной. События, отражающие действие «причин», в данном случае называют гипотезами, так как они — предполагаемые события, повлекшие данное.

$P(A|B) =\frac{P(A) \cdot P(B | A)}{P(B)}$ (1)

* $P(A)$  -- априорная вероятность гипотезы A;
* $P(A|B)$ --  вероятность гипотезы A при наступлении события B (апостериорная вероятность);
* $P(B|A)$ --  вероятность наступления события B при истинности гипотезы A;
* $P(B)$ -- полная вероятность наступления события B.





## Случай с одним признаком

Для классифкации на основе одного признака формулу (1) мы можем переписать следующим образом (2):

$P(class|feature) =\frac{P(class) \cdot P(feature | class)}{P(feature)}$ (2)

Различие между теоремой Байеса и наивным классификатором Байеса состоит в том, что наивный классификатор предполагает условную независимость, тогда как теорема Байеса ее не предполагает. Это означает, что между всеми входными свойствами классификатора нет взаимозависимости. 

Возможно это не очень удачное предположение, но именно поэтому этот алгоритм называют «наивным» (naive). В этом также одна из причин ускоренной работы алгоритма. Несмотря на то, что алгоритм «наивен», он все же может превзойти сложные модели. Поэтому не позволяйте названию вводить вас в заблуждение. 

In [21]:
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]


df = pd.DataFrame(data, columns=['weather', 'stroll'])
df

,weather,stroll
0,солнечно,True
1,снег,False
2,облачно,False
3,дождь,False
4,солнечно,True
5,снег,False
6,облачно,True
7,снег,False
8,солнечно,False
9,облачно,True


### Задание 1

Какова вероятность отправиться на прогулку если идёт дождь, при наличии следующих наблюдений?

Общая формула будет выглядеть так: $p(stroll|weather)=\frac{p(stroll)*p(weather|stroll)}{p(weather)}$
Применительно к конкретным значениям: $p(yes|rainy)=\frac{p(yes)*p(rainy|yes)}{p(rainy)}$

In [22]:
p_rainy = len(df.loc[df.weather == 'дождь']) / len(df)
p_stroll = len(df.loc[df.stroll == True]) / len(df)
p_rainy_if_stroll = len(df.loc[(df.stroll == True) & (df.weather == 'дождь')]) / len(df.loc[df.stroll == True])

p_stroll_if_rainy = p_stroll * p_rainy_if_stroll / p_rainy
p_stroll_if_rainy

0.3333333333333333

## Случай с несколькими дискретными признаками


В теореме Байеса вы должны вычислить единую условную вероятность с учетом всех свойств (вверху). С помощью наивного классификатора Байеса мы все упрощаем, вычисляя для каждого свойства условную вероятность, а затем перемножая их. Именно поэтому он и называется «наивным», поскольку условные вероятности всех свойств вычисляются независимо друг от друга.

Теорема Байеса:

$P(class | x_1, x_2, \dots, x_n) = \frac{P(x_1, x_2, \dots, x_n | class) \cdot P(A)}{P(x_1, x_2, \dots, x_n)}$ (5)

Naive Bayes:

$P(class | x_1, x_2, \dots, x_n) = p(class) \cdot P(x_1|class) \cdot P(x_2|class) \cdot ... \cdot P(x_n|class)$ (6)

### Задание 2

Разбить свои данные на тренировочную и тестовую выборку.
На основе обучающей создать модель наивного байеса, используя предложенное выше соотношение (6).

На тестовой выборке посчитать accuracy, precision и recall.

Если в данных содержатся числовые признаки, то разбейти их на квартили (границы: 25%, 50%, 75%  -- получится 4 бинарных признака).

In [23]:
a = np.array([1, 2, 1, 3])
np.unique(a, return_counts=True)

(array([1, 2, 3]), array([2, 1, 1], dtype=int64))

# Случай с числовыми признаками

Перед построением частотных таблиц числовые переменные можно преобразовать в их категориальные аналоги (разбиение). Другой вариант -- использовать распределение числовой переменной, чтобы иметь представление о её частоте. Например, одна из распространенных практик - предполагать нормальные распределения для числовых переменных.
 
Функция плотности вероятности для нормального распределения определяется двумя параметрами (средним значением и стандартным отклонением).

$\mu = \frac{1}{n}\sum\limits_{i=1}^{n} x_i$ -- среднее

$\sigma = \left[ \frac{1}{n-1} \sum\limits_{i=1}^{n} (x_i - \mu)^2 \right]^{0.5}$ -- стандартное отклонение

$P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i — \mu_y)^2}{2\sigma^2_y}\right)$ -- функция плотности для нормального распределения



In [24]:
class NaiveBayes:
    def __init__(self, is_continuous=True, quartile=4):
        self.is_continuous = is_continuous
        if self.is_continuous:
            self.mu = {}
            self.sigma = {}
        else:
            self.quartile = int(quartile)
            self.values_quartile = {}
            self.prob_quartile = {}
        self.classes = [0, 1]
        self.prob_classes = [0.5, 0.5]
    
    def fit(self, X, y, verbose=False):
        if self.is_continuous:
            self.mu = {}
            self.sigma = {}
        else:
            self.values_quartile = {}
            self.prob_quartile = {}
        classes, count_classes = np.unique(y, return_counts=True)
        self.classes = classes
        self.prob_classes = count_classes / len(y)
        for num_column in range(X.shape[1]):
            self.bayes_one_column_fit(num_column, X[:, num_column], y)
        if verbose and self.is_continuous:
            print(self.mu)
            print(self.sigma)
    
    def find_min_max(self, column):
        return column.min(), (column.max() - column.min()) / self.quartile
    
    def bayes_one_column_fit(self, name_column, column, y):
        column = column.astype("float")
        if self.is_continuous:
            mu_values = []
            sigma_values = []
            for y_class in self.classes:
                column_y_i = column[np.where(y == y_class)]
                mu_values.append(column_y_i.mean())
                sigma_values.append(column_y_i.std())
            self.mu[name_column] = mu_values
            self.sigma[name_column] = sigma_values
        else:
            min_val, step = self.find_min_max(column)
            vals_quartiles = [min_val + step * i for i in range(1, self.quartile)]
            self.values_quartile[name_column] = vals_quartiles
            prob_values = []
            for y_class in self.classes:
                column_y_i = column[np.where(y == y_class)]
                prev_vals = 0
                prob_values_y_i = []
                for val_quartile in vals_quartiles:
                    cur_vals = len(column_y_i[np.where(column_y_i <= val_quartile)])
                    prob_values_y_i.append((cur_vals - prev_vals) / len(column_y_i))
                    prev_vals = cur_vals
                prob_values_y_i.append((len(column_y_i) - prev_vals) / len(column_y_i))
                prob_values.append(prob_values_y_i)
            self.prob_quartile[name_column] = prob_values
    
    def density_func(self, x, mu, sigma, eps=1.0e-3):
        return np.exp(-np.power(x - mu, 2) / (2 * sigma * sigma + eps)) / np.sqrt(2. * np.pi * sigma * sigma + eps)
    
    def get_p_x_i(self, x, y_i):
        x = x.astype("float")
        for num_column in range(x.shape[0]):
            p_x_i = 1.
            if self.is_continuous:
                pass
            else:
                idx_x = 0
                for quartile in self.values_quartile[num_column]:
                    if x[num_column] < quartile:
                        break
                    else:
                        idx_x += 1
            if self.is_continuous:
                p_x_i *= self.density_func(x[num_column], self.mu[num_column][y_i], self.sigma[num_column][y_i])
            else:
                p_x_i *= self.prob_quartile[num_column][np.where(self.classes == y_i)[0][0]][idx_x]
        return p_x_i
                
    # P(class | x1...xn) = P(class) * P(x1|class) * ... * P(xn|class)
    def predict(self, X):
        return [
                self.classes[np.argmax([prob_class * self.get_p_x_i(x, y_i) for y_i, prob_class in zip(self.classes, self.prob_classes)])] 
            for x in X]

In [25]:
nb = NaiveBayes()
nb.fit(X_train_transform, y_train)
y_pred_CNB = nb.predict(X_test_transform)

In [26]:
nb = naive_bayes.GaussianNB()
nb.fit(X_train_transform, y_train)
y_pred_GNB = nb.predict(X_test_transform)

In [27]:
print_metrics(X_test_transform, y_test, y_pred_CNB)
print()
print_metrics(X_test_transform, y_test, y_pred_GNB)

Confusion matrix:
[[921  11]
 [872  52]]
Classification report:
              precision    recall  f1-score   support

           0       0.51      0.99      0.68       932
           1       0.83      0.06      0.11       924

    accuracy                           0.52      1856
   macro avg       0.67      0.52      0.39      1856
weighted avg       0.67      0.52      0.39      1856


Confusion matrix:
[[792 140]
 [395 529]]
Classification report:
              precision    recall  f1-score   support

           0       0.67      0.85      0.75       932
           1       0.79      0.57      0.66       924

    accuracy                           0.71      1856
   macro avg       0.73      0.71      0.71      1856
weighted avg       0.73      0.71      0.71      1856



In [28]:
X_1 = np.array([[0], [1], [0]])
y_1 = np.array([0, 1, 1])
X_2 = np.array([[0], [1]])
nb_2 = NaiveBayes(quartile=2)
nb_2.fit(X_1, y_1)
y_pred_CNB_1 = nb_2.predict(X_2)

nb_2_G = naive_bayes.GaussianNB()
nb_2_G.fit(X_1, y_1)
y_pred_GNB_1 = nb_2_G.predict(X_2)

In [29]:
print_metrics(X_2, np.array([0, 1]), y_pred_CNB_1)
print()
print_metrics(X_2, np.array([0, 1]), y_pred_GNB_1)

Confusion matrix:
[[1 0]
 [0 1]]
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2


Confusion matrix:
[[1 0]
 [0 1]]
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



# Вывод

В данной работе мы реализовали алгоритм Наивного Байесса.